In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2,
        rotation_range=60,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.3, # Randomly zoom image
        shear_range = 0.2,
        horizontal_flip=True,
        width_shift_range=0.122,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.122,  # randomly shift images vertically (fraction of total height)
                                  )

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1/255,validation_split=0.2)

In [ ]:
input_shape=(224,224,3)

In [ ]:
Training='/content/drive/MyDrive/fire_classification/'

In [ ]:
train_data = valid_datagen.flow_from_directory(
   Training,
    subset="training",
    seed=2,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    shuffle=True,
    class_mode="categorical")

Found 1148 images belonging to 2 classes.


In [ ]:
valid_data = valid_datagen.flow_from_directory(
   Training,
    subset="validation",
    seed=2,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode="categorical")

Found 286 images belonging to 2 classes.


In [ ]:
## imports for model
from tensorflow import keras
from keras.layers import Dropout, Flatten,Activation ,Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras.applications import ResNet50,DenseNet121,MobileNetV2


In [ ]:
conv_ResNet=DenseNet121(include_top=False,input_shape=input_shape)

29084464/29084464 [==============================] - 2s 0us/step


In [ ]:
conv_ResNet.trainable=False

In [ ]:
Early_Stopping = EarlyStopping(
    monitor='val_loss', # Same as the
    patience=5, # Wait for 5 epochs before stopping the training
    verbose=1,
    restore_best_weights='True', # Restore model weights from the epoch with the best value of the monitored quantity.
    min_delta=0.1
)
reduce_on_plateau =ReduceLROnPlateau(
        monitor  = 'loss',
        factor   = 0.1,
        patience = 2,
        verbose  = 1,
        mode     = 'min',
        epsilon  = 0.01,
        cooldown = 0,
        min_lr   = 0
    )

In [ ]:
inputs = tf.keras.Input(shape=input_shape)
x=conv_ResNet(inputs)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 512)               25690624  
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               131328

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
History = model.fit(train_data,validation_data =valid_data,epochs = 20,batch_size =32,callbacks=[reduce_on_plateau])

Epoch 1/20
36/36 [==============================] - 24s 677ms/step - loss: 0.0181 - accuracy: 0.9930 - val_loss: 0.1485 - val_accuracy: 0.9580 - lr: 1.0000e-05
Epoch 2/20
36/36 [==============================] - 28s 781ms/step - loss: 0.0108 - accuracy: 0.9974 - val_loss: 0.1493 - val_accuracy: 0.9580 - lr: 1.0000e-05
Epoch 3/20
36/36 [==============================] - ETA: 0s - loss: 0.0194 - accuracy: 0.9956
Epoch 3: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
36/36 [==============================] - 24s 657ms/step - loss: 0.0194 - accuracy: 0.9956 - val_loss: 0.1480 - val_accuracy: 0.9580 - lr: 1.0000e-05
Epoch 4/20
36/36 [==============================] - 24s 649ms/step - loss: 0.0109 - accuracy: 0.9983 - val_loss: 0.1485 - val_accuracy: 0.9580 - lr: 1.0000e-06
Epoch 5/20
36/36 [==============================] - ETA: 0s - loss: 0.0124 - accuracy: 0.9956
Epoch 5: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
36/36 [======================

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/MyDrive/dense_fire.h5')